In [1]:
import numpy as np
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import keyring
import time
import os

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id=keyring.get_password('spotify', 'cid'),
                                                      client_secret=keyring.get_password('spotify', 'secret') )
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
KEYWORD = 'Tiktok'

In [6]:
results = sp.search(q=KEYWORD, type='playlist', market='US')

In [7]:
results

{'playlists': {'href': 'https://api.spotify.com/v1/search?query=Tiktok&type=playlist&market=US&offset=0&limit=10',
  'items': [{'collaborative': False,
    'description': 'Viral, trending and taking off.',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX2L0iB23Enbq'},
    'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX2L0iB23Enbq',
    'id': '37i9dQZF1DX2L0iB23Enbq',
    'images': [{'height': None,
      'url': 'https://i.scdn.co/image/ab67706f00000003d971c6c23114fc7636dc23eb',
      'width': None}],
    'name': 'Viral Hits',
    'owner': {'display_name': 'Spotify',
     'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
     'href': 'https://api.spotify.com/v1/users/spotify',
     'id': 'spotify',
     'type': 'user',
     'uri': 'spotify:user:spotify'},
    'primary_color': None,
    'public': None,
    'snapshot_id': 'MTY2NDUxMDQ2MCwwMDAwMDAwMDc4OWU4OTFhMWE2ODAxZTgxOGY3Y2QzMDdlYTUwYmU0',
    'tracks': {'href': 'https://api

In [8]:
[r['name'] for r in results['playlists']['items']]

['Viral Hits',
 'TikTok Songs 2022 | Tik Tok Hits | Viral Hits ',
 'TikTok songs you can’t get out of your head | 2022 TikTok songs ',
 'Sad tiktok audios',
 'ava<3',
 'tiktok october 2022',
 'Ava<3',
 'TikTok Phonk Music 2022',
 'Sigma Male Music {TikTok}',
 'TikTok songs <3']

In [9]:
results['playlists'].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [10]:
playlist_ids = []
playlist_names = []
playlist_numtracks = []

N = 100
#get playlist in batches of 50
for n in np.arange(N//50):
    offset= 50*n
    print("Getting batch %d of search results for keyword: %s ..." % (n,KEYWORD), end='' )
    results = sp.search(q=KEYWORD, type='playlist' , market='US', offset = offset, limit=50)
    playlist_ids.extend([p['href'].split('/')[5] for p in results['playlists']['items']])
    playlist_names.extend([p['name'] for p in results['playlists']['items']])
    playlist_numtracks.extend([p['tracks']['total'] for p in results['playlists']['items']])
    print("  DONE!")

Getting batch 0 of search results for keyword: Tiktok ...  DONE!
Getting batch 1 of search results for keyword: Tiktok ...  DONE!


In [11]:
playlist_lookup = []
for n,p_id in enumerate(playlist_ids):
    print("Getting playlist data for playlist %s :..." % (playlist_names[n]), end='' )
    playlist = sp.playlist(p_id)
    try:
        relevant_playlist_data = { key: playlist[key] for key in ['followers','owner']}
        relevant_playlist_data['playlist_id'] = p_id
        relevant_playlist_data['playlist_name'] = playlist_names[n]
        relevant_playlist_data['playlist_total_tracks'] = playlist_numtracks[n]
        relevant_playlist_data['owner_id'] = playlist['owner']['id']
        relevant_playlist_data['owner_name'] = playlist['owner']['display_name']
        relevant_playlist_data['total_followers'] = playlist['followers']['total']
        relevant_playlist_data.pop('owner', None)
        relevant_playlist_data.pop('followers', None)
        playlist_lookup.append(relevant_playlist_data)
        print("   DONE")
    except:
        print("   Aborted")
        continue

Getting playlist data for playlist Viral Hits :...   DONE
Getting playlist data for playlist TikTok Songs 2022 | Tik Tok Hits | Viral Hits  :...   DONE
Getting playlist data for playlist TikTok songs you can’t get out of your head | 2022 TikTok songs  :...   DONE
Getting playlist data for playlist Sad tiktok audios :...   DONE
Getting playlist data for playlist ava<3 :...   DONE
Getting playlist data for playlist tiktok october 2022 :...   DONE
Getting playlist data for playlist Ava<3 :...   DONE
Getting playlist data for playlist TikTok Phonk Music 2022 :...   DONE
Getting playlist data for playlist Sigma Male Music {TikTok} :...   DONE
Getting playlist data for playlist TikTok songs <3 :...   DONE
Getting playlist data for playlist TikTok 洋楽 (絶対知ってる) :...   DONE
Getting playlist data for playlist Tiktok edit audios 2022 :...   DONE
Getting playlist data for playlist Sped up TikTok Songs 2022 :...   DONE
Getting playlist data for playlist tiktok remixes/ edit sounds :) :...   DONE
Get

In [12]:
playlist_df = pd.DataFrame(playlist_lookup)
playlist_df = playlist_df.sort_values('total_followers',ascending=False)
playlist_df 

,playlist_id,playlist_name,playlist_total_tracks,owner_id,owner_name,total_followers
66,37i9dQZF1DX5Vy6DFOcx00,big on the internet,133,spotify,Spotify,2536421
0,37i9dQZF1DX2L0iB23Enbq,Viral Hits,106,spotify,Spotify,2324429
1,65LdqYCLcsV0lJoxpeQ6fW,TikTok Songs 2022 | Tik Tok Hits | Viral Hits,300,43xzlwfo0tft7hn5r5e6dksrd,LoudKult,1927993
15,57EG9lWmdn7HHofXuQVsow,TikTok Music 2022 | TikTok Songs | TikTok Hits,200,21ifzjxggdajdpc4w6iqxslkq,Soave,1271125
2,0JFatPoPq82gNcPa4esOzj,TikTok songs you can’t get out of your head | ...,101,filtr,Filtr,1070515
...,...,...,...,...,...,...
63,1KFM3rbqqhEWlFaPRKl1ZD,Tiktok Dance craze 2022,91,22kuhsh2fgvyzll2psyduqdia,CJ,533
21,5OJ1qROBZMx8bBKvR6qA9K,Those vent tiktok videos,58,31p7nl2onlplpwukmenpcjmdx5u4,DOUBLE HAT PERSON,450
58,7x5JhqdQjvCM0FknM9VilX,TikTok Internacionais 2022 🔥 TikTok 2022 Novas,96,filtr.br,Filtr Brasil\t,388
4,5b6UtBk5uxIRNFfQAgNXxt,ava<3,24,31n6xdxmwwobfxhg7iuhixkxi3ka,geo,249


In [13]:
playlist_df = playlist_df[playlist_df['playlist_name'].str.lower().str.contains(KEYWORD.lower())]

In [14]:
playlist_df

,playlist_id,playlist_name,playlist_total_tracks,owner_id,owner_name,total_followers
1,65LdqYCLcsV0lJoxpeQ6fW,TikTok Songs 2022 | Tik Tok Hits | Viral Hits,300,43xzlwfo0tft7hn5r5e6dksrd,LoudKult,1927993
15,57EG9lWmdn7HHofXuQVsow,TikTok Music 2022 | TikTok Songs | TikTok Hits,200,21ifzjxggdajdpc4w6iqxslkq,Soave,1271125
2,0JFatPoPq82gNcPa4esOzj,TikTok songs you can’t get out of your head | ...,101,filtr,Filtr,1070515
71,0uqvdRxLapFl6omBnkkLx9,slowed + reverbed tiktok songs 2022,274,xbdq7aotyh862s0xlry2kzndo,Relaxing Sounds,467154
12,3HGejOHMRNkjZgjeH2W3UG,Sped up TikTok Songs 2022,152,hrsdhaeyyirqp9qj8j52ibl3s,MusicUp2Date,204042
...,...,...,...,...,...,...
99,1b6ozUuarVVk0TQYwaTmUE,TikTok Gym Edit Audios,298,12163768380,Angelo,746
82,2JXzb6fZPoUifeCXR6Js24,"TIKTOK PHONK ( TikTok phonk edit audio, edit a...",23,31nlivfafazr2ozrzr3eizikchje,sashimi,668
63,1KFM3rbqqhEWlFaPRKl1ZD,Tiktok Dance craze 2022,91,22kuhsh2fgvyzll2psyduqdia,CJ,533
21,5OJ1qROBZMx8bBKvR6qA9K,Those vent tiktok videos,58,31p7nl2onlplpwukmenpcjmdx5u4,DOUBLE HAT PERSON,450


In [15]:
playlist_df.to_csv("data/"+KEYWORD+"_playlist_data.csv",encoding='utf=8',index=False)

In [16]:
def get_relevant_track_data(tracks_data, playlist_id, playlist_name):
    try:
        relevant_track_data = { key: tracks_data['track'][key] for key in ['id','artists','name','popularity','duration_ms'] }
        relevant_track_data['artist_id']=[artist['id'] for artist in relevant_track_data['artists'] ]
        relevant_track_data['artist_name']=[artist['name']for artist in relevant_track_data['artists'] ]
        relevant_track_data['num_artists']=len([artist['id'] for artist in relevant_track_data['artists']]) 
        relevant_track_data['playlist_id']=playlist_id
        relevant_track_data['playlist_name']=playlist_name
        
        relevant_track_data.pop('artists', None)
        return relevant_track_data
    except:
        return 

In [17]:
#playlist_tracks
all_track_data = []
time.sleep(10)
for _,p_id,p_name, p_numtracks in playlist_df[['playlist_id','playlist_name','playlist_total_tracks']].to_records():
    print("Fetching data for playlist : %s(%s), with total tracks: %d" % (p_name,p_id,p_numtracks))
    n_fetches = p_numtracks // 100
    
    playlist_track_data = []
    #get tracks in batches of 100
    for n in np.arange(n_fetches+1):
        track_data = sp.playlist_tracks(p_id, offset=n*100)
        playlist_track_data.extend([get_relevant_track_data(item, p_id,p_name) for item in track_data['items']])
        
    all_track_data.extend(playlist_track_data)

Fetching data for playlist : TikTok Songs 2022 | Tik Tok Hits | Viral Hits (65LdqYCLcsV0lJoxpeQ6fW), with total tracks: 300
Fetching data for playlist : TikTok Music 2022 | TikTok Songs | TikTok Hits(57EG9lWmdn7HHofXuQVsow), with total tracks: 200
Fetching data for playlist : TikTok songs you can’t get out of your head | 2022 TikTok songs (0JFatPoPq82gNcPa4esOzj), with total tracks: 101
Fetching data for playlist : slowed + reverbed tiktok songs 2022(0uqvdRxLapFl6omBnkkLx9), with total tracks: 274
Fetching data for playlist : Sped up TikTok Songs 2022(3HGejOHMRNkjZgjeH2W3UG), with total tracks: 152
Fetching data for playlist : tiktok remixes / viral tiktok sounds (for edit)(0TSlZBUJACjsdyptBU7AYQ), with total tracks: 91
Fetching data for playlist : TikTok & Triller Hits(7ir419vfZESRBL6QBh0A0f), with total tracks: 35
Fetching data for playlist : Sped up songs 2022 - Speed up audios speed up songs - sped up tiktok songs - sped version(4sQRT2zDzaMk92Fw75TZHn), with total tracks: 100
Fetch

Fetching data for playlist : tiktok love songs 2022(4qfKXdLwk7ruT8uM6sdVXq), with total tracks: 156
Fetching data for playlist : Top Hits Lagu Barat Viral TikTok(61oEALt4q4knkIgMg6l4C6), with total tracks: 40
Fetching data for playlist : A TRAIN TIKTOK(1XRGy9ZBFmIzi4igGHfeY3), with total tracks: 53
Fetching data for playlist : SOCCER TIKTOK MUSIC❤️‍🔥❤️‍🔥⚽️⚽️(66bZau1gyVVsapUAfgmT9u), with total tracks: 32
Fetching data for playlist : tiktok songs october 2022(4SKXHjXwNPydxW4MMgwMT3), with total tracks: 47
Fetching data for playlist : TIKTOK dances(1uqg7LKZoxJMmIBsPlf18z), with total tracks: 125
Fetching data for playlist : TikTok Songs (2019-2020-2021) (23yXL9lMv6F4XDIvqClH17), with total tracks: 585
Fetching data for playlist : it’s corn (tiktok song) (1kBQA4xeI7mhw7jKXAY3wY), with total tracks: 44
Fetching data for playlist : real. (tiktok slideshows)(5J9AwRN29QJunf6LQFbtam), with total tracks: 42
Fetching data for playlist : you're in a tiktok edit(7fIKkNHZNF2fF4HCTT9tsQ), with total

In [18]:
for n,a in enumerate(all_track_data):
    try:
        len(a)
    except:
        print(n)

672
673
674
675
676
677
678
679
689
690
691
692
693
694
695
696
697
698
699
700
713
714
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
875
876
877
878
1028
1153
1166
1200
1219
2384
2455
2459
2633
2844
3440
3671
4712
4829
5020
5034
5054
5056
5063
5064
5065
5066
5089
5093
5149
5210
5213
5219
5258
5260
5269
5334
5672
5673
5674
5680
5681
5682
5686
5698
5702
5722
5725
5726
5729
5732
5735
5772
5773
5774
5775
5784
5787
5788
5789
5790
5798
5824
5827
5828
5829
5830
5831
5832
5833
5834
5836
5838
5843
5844
5863
6301
6302
6476
6592
6679
6680
7805
7817
7829
7837
7973
7974
9647
9715
9729
10423
10424
10501
10543
10566
10865
10881
10949
10995
11428
11582
11583
11615
12280
12408
12740


In [19]:
tracks_df = pd.DataFrame([data for data in all_track_data if data is not None])
tracks_df = tracks_df.rename(columns={'id':'track_id'})
tracks_df['artist_id'] = tracks_df.apply(lambda x: x['artist_id'][0] if x['num_artists']==1 else x['artist_id'], axis=1)
tracks_df['artist_name'] = tracks_df.apply(lambda x: x['artist_name'][0] if x['num_artists']==1 else x['artist_name'], axis=1)
tracks_df.head()

,track_id,name,popularity,duration_ms,artist_id,artist_name,num_artists,playlist_id,playlist_name
0,4C6Uex2ILwJi9sZXRdmqXp,Super Freaky Girl,93,170977,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,1,65LdqYCLcsV0lJoxpeQ6fW,TikTok Songs 2022 | Tik Tok Hits | Viral Hits
1,3RfTeIrWS8LNrfHYwechtt,Sunroof,30,163025,"[7qmpXeNz2ojlMl2EEfkeLs, 38PzLQE4GW8o7A18oGhi0x]","[Nicky Youre, dazy]",2,65LdqYCLcsV0lJoxpeQ6fW,TikTok Songs 2022 | Tik Tok Hits | Viral Hits
2,27ZZdyTSQWI7Cug2d2PkqV,BILLIE EILISH.,89,99282,2qAwMsiIjTzlmfAkXKvhVA,Armani White,1,65LdqYCLcsV0lJoxpeQ6fW,TikTok Songs 2022 | Tik Tok Hits | Viral Hits
3,4LRPiXqCikLlN15c3yImP7,As It Was,96,167303,6KImCVD70vtIoJWnq6nGn3,Harry Styles,1,65LdqYCLcsV0lJoxpeQ6fW,TikTok Songs 2022 | Tik Tok Hits | Viral Hits
4,3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),96,156943,"[2wY79sveU1sp5g7SokKOiI, 3Xt3RrJMFv5SZkCfUE8C1J]","[Sam Smith, Kim Petras]",2,65LdqYCLcsV0lJoxpeQ6fW,TikTok Songs 2022 | Tik Tok Hits | Viral Hits


In [20]:
tracks_df.to_csv("data/"+KEYWORD+"_playlist_tracks.csv",encoding='utf=8',index=False)

In [21]:
tracks_df = pd.read_csv("data/"+KEYWORD+"_playlist_tracks.csv")
tracks_df.head()

,track_id,name,popularity,duration_ms,artist_id,artist_name,num_artists,playlist_id,playlist_name
0,4C6Uex2ILwJi9sZXRdmqXp,Super Freaky Girl,93,170977,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,1,65LdqYCLcsV0lJoxpeQ6fW,TikTok Songs 2022 | Tik Tok Hits | Viral Hits
1,3RfTeIrWS8LNrfHYwechtt,Sunroof,30,163025,"['7qmpXeNz2ojlMl2EEfkeLs', '38PzLQE4GW8o7A18oG...","['Nicky Youre', 'dazy']",2,65LdqYCLcsV0lJoxpeQ6fW,TikTok Songs 2022 | Tik Tok Hits | Viral Hits
2,27ZZdyTSQWI7Cug2d2PkqV,BILLIE EILISH.,89,99282,2qAwMsiIjTzlmfAkXKvhVA,Armani White,1,65LdqYCLcsV0lJoxpeQ6fW,TikTok Songs 2022 | Tik Tok Hits | Viral Hits
3,4LRPiXqCikLlN15c3yImP7,As It Was,96,167303,6KImCVD70vtIoJWnq6nGn3,Harry Styles,1,65LdqYCLcsV0lJoxpeQ6fW,TikTok Songs 2022 | Tik Tok Hits | Viral Hits
4,3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),96,156943,"['2wY79sveU1sp5g7SokKOiI', '3Xt3RrJMFv5SZkCfUE...","['Sam Smith', 'Kim Petras']",2,65LdqYCLcsV0lJoxpeQ6fW,TikTok Songs 2022 | Tik Tok Hits | Viral Hits


In [22]:
def get_track_data(t_id, playlist_id,playlist_name):                    
    track_data = sp.track(t_id)
    track_features = sp.audio_features(t_id)
    
    #get only main(first) artist
    td_list = [t_id,\
               track_data['name'],\
               track_data['artists'][0]['id'],\
               track_data['artists'][0]['name'],\
               track_data['album']['uri'].split(":")[2],\
               track_data['duration_ms'],\
               track_data['album']['release_date'],\
               track_data['popularity']]
    data = pd.DataFrame([td_list], columns = ['track_id','track_name','artist_id','artist_name','album_id',\
                                              'duration','release_date','popularity'])

    relevant_cols = ['danceability', 'energy', 'key', 'loudness', 'mode',\
                     'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']  
    
    tf_data = pd.DataFrame(track_features)
    tf_data = tf_data[relevant_cols]
    #tag with source playlist
    tf_data['playlist_id'] = playlist_id
    tf_data['playlist_name'] = playlist_name
    
    data = pd.concat([data, tf_data], axis=1)
    return data


In [23]:
downloaded_track_data = []

In [24]:
track_list = tracks_df['track_id'].values
playlist_name_list = tracks_df['playlist_name'].values
playlist_id_list = tracks_df['playlist_id'].values
df_list=[]

for i,track_id in enumerate(track_list):
    try:
        if track_id not in downloaded_track_data:
            print('[%d/%d] Fetching track data for %s... ' % 
                  (i+1,len(track_list),tracks_df[tracks_df['track_id']==track_id]['name'].values[0]), end = " ") 
            track_data = get_track_data(track_id, playlist_id_list[i],playlist_name_list[i]) 
            df_list.append(track_data)
            downloaded_track_data.append(track_id)
            print('done!')
    except:
        continue
    else:
        continue
    
    #sleep for 20 secs per 70 requests to avoid being blocked
    if (i % 70 == 0)&(i > 0):
        time.sleep(20)    

[1/12695] Fetching track data for Super Freaky Girl...  done!
[2/12695] Fetching track data for Sunroof...  done!
[3/12695] Fetching track data for BILLIE EILISH....  done!
[4/12695] Fetching track data for As It Was...  done!
[5/12695] Fetching track data for Unholy (feat. Kim Petras)...  done!
[6/12695] Fetching track data for I'm Good (Blue)...  done!
[7/12695] Fetching track data for About Damn Time...  done!
[8/12695] Fetching track data for Glimpse of Us...  done!
[9/12695] Fetching track data for Doja...  done!
[10/12695] Fetching track data for Running Up That Hill (A Deal With God)...  done!
[11/12695] Fetching track data for Heat Waves...  done!
[12/12695] Fetching track data for Bad Habit...  done!
[13/12695] Fetching track data for Baby Shark...  done!
[14/12695] Fetching track data for STAY (with Justin Bieber)...  done!
[15/12695] Fetching track data for Romantic Homicide...  done!
[16/12695] Fetching track data for I Ain't Worried...  done!
[17/12695] Fetching track data

[135/12695] Fetching track data for Paradise...  done!
[136/12695] Fetching track data for Buss It...  done!
[137/12695] Fetching track data for Apologize...  done!
[138/12695] Fetching track data for Beauty And A Beat...  done!
[139/12695] Fetching track data for Why...  done!
[140/12695] Fetching track data for Better Days (NEIKED x Mae Muller x Polo G)...  done!
[141/12695] Fetching track data for The Motto...  done!
[142/12695] Fetching track data for Need to Know...  done!
[143/12695] Fetching track data for Pompeii...  done!
[144/12695] Fetching track data for Whatever You Like...  done!
[145/12695] Fetching track data for MONEY...  done!
[146/12695] Fetching track data for Freaks...  done!
[147/12695] Fetching track data for Tainted Love...  done!
[148/12695] Fetching track data for Heart of Glass...  done!
[149/12695] Fetching track data for Blinding Lights...  done!
[150/12695] Fetching track data for Thot Shit...  done!
[151/12695] Fetching track data for Pumped Up Kicks...  

[267/12695] Fetching track data for Falling...  done!
[268/12695] Fetching track data for Let It Go...  done!
[269/12695] Fetching track data for death bed (coffee for your head)...  done!
[270/12695] Fetching track data for That Way...  done!
[271/12695] Fetching track data for Baby I'm Yours...  done!
[272/12695] Fetching track data for BOP...  done!
[273/12695] Fetching track data for OUT WEST (feat. Young Thug)...  done!
[274/12695] Fetching track data for Blinding Lights...  done!
[275/12695] Fetching track data for Skechers...  done!
[276/12695] Fetching track data for ROXANNE...  done!
[277/12695] Fetching track data for Shooting Stars...  done!
[278/12695] Fetching track data for The Git Up...  done!
[279/12695] Fetching track data for Payphone...  done!
[280/12695] Fetching track data for I Don't Care (with Justin Bieber)...  done!
[281/12695] Fetching track data for hot girl bummer...  done!
[282/12695] Fetching track data for Demons...  done!
[283/12695] Fetching track data 

[449/12695] Fetching track data for If You Had My Love (Kav Verhouzer Remix)...  done!
[451/12695] Fetching track data for Castaway...  done!
[454/12695] Fetching track data for No Role Modelz...  done!
[455/12695] Fetching track data for Options...  done!
[456/12695] Fetching track data for traitor...  done!
[457/12695] Fetching track data for Soldja...  done!
[458/12695] Fetching track data for Monëy so big...  done!
[459/12695] Fetching track data for Moët Stained Margiela...  done!
[461/12695] Fetching track data for Overdose...  done!
[463/12695] Fetching track data for Spaceship...  done!
[465/12695] Fetching track data for Not The One...  done!
[467/12695] Fetching track data for Unhappy Ever After...  done!
[469/12695] Fetching track data for Cravin'...  done!
[470/12695] Fetching track data for Moscow Mule...  done!
[471/12695] Fetching track data for Money...  done!
[472/12695] Fetching track data for bad guy...  done!
[473/12695] Fetching track data for not the same...  done

[622/12695] Fetching track data for Show Me Your Back - Slowed + Reverb...  done!
[623/12695] Fetching track data for SHADOW - Slowed + Reverb...  done!
[624/12695] Fetching track data for MUSHROOMS...  done!
[625/12695] Fetching track data for Dream Girl - Slowed + Reverb...  done!
[626/12695] Fetching track data for Scary Garry - Sped Up...  done!
[627/12695] Fetching track data for BREAKING POINT - Slowed + Reverb...  done!
[628/12695] Fetching track data for AVOID ME - Slowed + Reverb...  done!
[629/12695] Fetching track data for OLD GENESIS - Remix - Slowed + Reverb...  done!
[630/12695] Fetching track data for YAYO - Slowed + Reverb...  done!
[631/12695] Fetching track data for just for me...  done!
[632/12695] Fetching track data for RAVE - Slowed...  done!
[633/12695] Fetching track data for Bow - Slowed...  done!
[634/12695] Fetching track data for Scary Garry - Slowed + Reverb...  done!
[635/12695] Fetching track data for Razor Bones...  done!
[636/12695] Fetching track data 

...

[11762/12695] Fetching track data for Bandaid On A Bullet Hole...  done!
[11763/12695] Fetching track data for The Way I Talk...  done!
[11764/12695] Fetching track data for Rumor...  done!
[11765/12695] Fetching track data for Six Feet Apart...  done!
[11766/12695] Fetching track data for Drunk Driving...  done!
[11767/12695] Fetching track data for Must've Never Met You...  done!
[11768/12695] Fetching track data for Southern Land...  done!
[11769/12695] Fetching track data for One Man Band...  done!
[11770/12695] Fetching track data for UNAPOLOGETICALLY COUNTRY AS HELL...  done!
[11771/12695] Fetching track data for It's Five O'Clock Somewhere...  done!
[11772/12695] Fetching track data for Don't Rock the Jukebox...  done!
[11773/12695] Fetching track data for All My Ex's Live In Texas - Live...  done!
[11774/12695] Fetching track data for Brand New Man - with Luke Combs...  done!
[11775/12695] Fetching track data for Should've Been A Cowboy...  done!
[11776/12695] Fetching track da

[11883/12695] Fetching track data for February 28, 2016...  done!
[11884/12695] Fetching track data for This Damn Song...  done!
[11885/12695] Fetching track data for Brokenheartsville...  done!
[11886/12695] Fetching track data for Pretty Heart...  done!
[11887/12695] Fetching track data for Something to Talk About...  done!
[11888/12695] Fetching track data for Teenage Dream...  done!
[11889/12695] Fetching track data for Sunshine Girl...  done!
[11890/12695] Fetching track data for Neon Moon...  done!
[11891/12695] Fetching track data for Down the Road...  done!
[11892/12695] Fetching track data for Up Down (feat. Florida Georgia Line)...  done!
[11893/12695] Fetching track data for Cowboy...  done!
[11894/12695] Fetching track data for Strawberry Wine...  done!
[11895/12695] Fetching track data for Tattoos...  done!
[11896/12695] Fetching track data for Wild as You...  done!
[11897/12695] Fetching track data for To Be Loved By You...  done!
[11898/12695] Fetching track data for Thi

[12008/12695] Fetching track data for Na Na Na (Na Na Na Na Na Na Na Na Na)...  done!
[12009/12695] Fetching track data for Dead!...  done!
[12010/12695] Fetching track data for Boot Scootin' Boogie...  done!
[12011/12695] Fetching track data for Save a Horse (Ride a Cowboy)...  done!
[12012/12695] Fetching track data for Red Solo Cup...  done!
[12013/12695] Fetching track data for Be My Baby Tonight...  done!
[12014/12695] Fetching track data for Any Man Of Mine...  done!
[12016/12695] Fetching track data for Can't Dance...  done!
[12017/12695] Fetching track data for Give It Away...  done!
[12018/12695] Fetching track data for Red Dirt Road...  done!
[12019/12695] Fetching track data for If I Didn’t Wear Boots...  done!
[12020/12695] Fetching track data for When She Comes Home Tonight...  done!
[12021/12695] Fetching track data for Drive (For Daddy Gene)...  done!
[12022/12695] Fetching track data for I Drive Your Truck...  done!
[12023/12695] Fetching track data for Fall In Love... 

[12190/12695] Fetching track data for Отменяй...  done!
[12191/12695] Fetching track data for Miss The Rage Reversed...  done!
[12192/12695] Fetching track data for help_urself - boosted...  done!
[12193/12695] Fetching track data for The Perfect Girl - (Instrumental) [Slowed]...  done!
[12199/12695] Fetching track data for Driver Forever (Slowed)...  done!
[12200/12695] Fetching track data for Hah...  done!
[12201/12695] Fetching track data for Off the Leash...  done!
[12207/12695] Fetching track data for little dark age - little dark age...  done!
[12208/12695] Fetching track data for I Found Amongus...  done!
[12209/12695] Fetching track data for Homage - Slowed+Reverb...  done!
[12211/12695] Fetching track data for Money So Big...  done!
[12212/12695] Fetching track data for Miss The Rage - Instrumental...  done!
[12213/12695] Fetching track data for Suffer with Me...  done!
[12216/12695] Fetching track data for Neva 2 Much Instrumental - slowed+reverb...  done!
[12217/12695] Fetch

[12498/12695] Fetching track data for TWINNEM...  done!
[12499/12695] Fetching track data for Missing My Love...  done!
[12506/12695] Fetching track data for Like That Slowed - Remix...  done!
[12508/12695] Fetching track data for All I Wanna Do...  done!
[12514/12695] Fetching track data for Ex For A Reason (with JT from City Girls)...  done!
[12517/12695] Fetching track data for I Got a Man But I Want You...  done!
[12520/12695] Fetching track data for From Tha Back...  done!
[12521/12695] Fetching track data for All About Him...  done!
[12522/12695] Fetching track data for Nasty...  done!
[12524/12695] Fetching track data for Lay It Down (Remix) (feat. Lloyd)...  done!
[12525/12695] Fetching track data for Wild Side (feat. Cardi B)...  done!
[12526/12695] Fetching track data for I Don't Want Her (feat. Problem)...  done!
[12533/12695] Fetching track data for Before I Die/Play No Games...  done!
[12537/12695] Fetching track data for You Right X Luxurious...  done!
[12541/12695] Fetch